# Late blight prediction for the Columbia Basin of Washington

# Date: 3/25/2020

# Authors:
- Original models were developed by Dennis Johnson and colleagues:
    - Johnson DA, Alldredge JR, and Vakoch DL. 1996. Potato late blight forecasting models for the semiarid environment of south-central Washington. Phytopathology 86:480-484. https://www.apsnet.org/publications/phytopathology/backissues/Documents/1996Articles/Phyto86n05_480.PDF
    - Johnson DA, Alldredge JR, and Hamm PB. 1998. Expansion of potato late blight forecasting models for the Columbia Basin of Washington and Oregon. Plant Dis. 82:642-645. https://apsjournals.apsnet.org/doi/pdfplus/10.1094/PDIS.1998.82.6.642
    - Johnson DA, Cummings TF, Abi Ghanem R, and Alldredge JR. 2009. Association of solar irradiance and days of precipitation with incidence of potato late blight in the semiarid environment of the Columbia Basin. Plant Dis. 93:272-280. https://apsjournals.apsnet.org/doi/pdfplus/10.1094/PDIS-93-3-0272
    - Johnson DA, and Cummings TF. 2016. In-canopy environment of sprinkler irrigated potato fields as a factor for late blight management in the semiarid environment of the Columbia Basin. Am J. Potato Res: 93:239-252 https://link.springer.com/article/10.1007/s12230-016-9500-1


- This script was written by David Linnard Wheeler | david.wheeler@wsu.edu | dlinnard@berkeley.edu | office: 509.335.3733 | cell: 215.880.3024
       

# Objectives

**Predict late blight epidemics in the Columbia Basin of Washington State**

# Risk

**Risk of late blight is the product of 5 factors**

**Risk** = **A** (phenology) $\cdot$ **B** (occurrence of late blight in field or adjacent fields) $\cdot$ **C** (probability of late blight occurrence in the Columbia Basin (PROB)) $\cdot$ **D** (date) $\cdot$ **E** (number of rainy days expected in the next 7 days)

### A (phenology)

**Source of information: growers**

| Factor level | Factor value |
| --- | --- |
| Pre-emergence | 0 |
| Emergence to before plant closure between rows | 0 |
| Plant closure between rows (foliage touching between adjacent rows) | 1 |
|Post row closure to harvest | 1 |

### B (occurrence of late blight in field or adjacent fields within 5 mile radius)

**Source of information: growers**

| Factor level | Factor value |
| --- | --- |
| No | 0 |
| Yes | 30 |

### C (probability of late blight occurrence in the Columbia Basin (PROB))

#### Import data for rain in April

Packages and modules

In [2]:
import pandas as pd
import numpy as np

Prosser data

In [3]:
ApPrDF = pd.read_csv('/Users/davidwheeler/Desktop/RESEARCH/Data/LateBightForecast/Forecasts/30DaySpotForecasts/AprilMayRain2020/AprilRain2020/ProsserApril_2020.csv')  

Summary

In [4]:
ApPrDF.shape

(30, 18)

In [5]:
ApPrDF.head()

,Date,Date.1,Min°F,Avg°F,Max°F,AvgDP°F,AvgRH%,AvgLWu.,AvgDir°,AvgSpeedmph,MaxGustmph,Avg 2 in.°F,Min°F.1,Avg°F.1,TotPrecin,TotalSolarRadMJ/m²,EToin,ETrin
0,2020/04/01,1,30.1,40.2,49.9,27.8,62.0,0.01,W,4.1,17.1,NaN,48.0,49.7,0.0,13.57,0.08,0.10
1,2020/04/02,2,26.6,40.2,53.0,25.3,59.0,0.02,SW,4.5,22.5,NaN,46.6,49.0,0.0,19.42,0.10,0.13
2,2020/04/03,3,26.9,40.5,54.1,21.1,49.8,0.00,W,6.1,31.4,NaN,46.8,49.3,0.0,20.68,0.12,0.17
3,2020/04/04,4,29.7,42.5,53.4,22.4,47.6,0.01,NE,4.8,24.3,NaN,47.1,49.1,0.0,16.25,0.10,0.14
4,2020/04/05,5,38.0,47.3,58.3,32.7,58.0,0.00,NE,3.9,18.9,NaN,49.4,51.0,0.0,16.26,0.10,0.14


In [6]:
ApPrDF['Station'] = "Prosser"

In [7]:
ApPrDF.head()

,Date,Date.1,Min°F,Avg°F,Max°F,AvgDP°F,AvgRH%,AvgLWu.,AvgDir°,AvgSpeedmph,MaxGustmph,Avg 2 in.°F,Min°F.1,Avg°F.1,TotPrecin,TotalSolarRadMJ/m²,EToin,ETrin,Station
0,2020/04/01,1,30.1,40.2,49.9,27.8,62.0,0.01,W,4.1,17.1,NaN,48.0,49.7,0.0,13.57,0.08,0.10,Prosser
1,2020/04/02,2,26.6,40.2,53.0,25.3,59.0,0.02,SW,4.5,22.5,NaN,46.6,49.0,0.0,19.42,0.10,0.13,Prosser
2,2020/04/03,3,26.9,40.5,54.1,21.1,49.8,0.00,W,6.1,31.4,NaN,46.8,49.3,0.0,20.68,0.12,0.17,Prosser
3,2020/04/04,4,29.7,42.5,53.4,22.4,47.6,0.01,NE,4.8,24.3,NaN,47.1,49.1,0.0,16.25,0.10,0.14,Prosser
4,2020/04/05,5,38.0,47.3,58.3,32.7,58.0,0.00,NE,3.9,18.9,NaN,49.4,51.0,0.0,16.26,0.10,0.14,Prosser


TriCities data

In [8]:
ApTCDF = pd.read_csv('/Users/davidwheeler/Desktop/RESEARCH/Data/LateBightForecast/Forecasts/30DaySpotForecasts/AprilMayRain2020/AprilRain2020/TriCitiesApril_2020.csv')  

In [9]:
ApTCDF['Station'] = "TriCities"

In [10]:
ApTCDF.head()

,Date,Date.1,Min°F,Avg°F,Max°F,AvgDP°F,AvgRH%,AvgLWu.,AvgDir°,AvgSpeedmph,MaxGustmph,Avg 2 in.°F,Min°F.1,Avg°F.1,TotPrecin,TotalSolarRadMJ/m²,EToin,ETrin,AvgAtm.PressinHg,Station
0,2020/04/01,1,34.6,43.2,52.3,29.7,59.9,0.02,SW,4.5,14.2,NaN,50.8,51.8,0.0,10.53,0.08,0.11,30.06,TriCities
1,2020/04/02,2,25.4,41.8,54.9,24.5,54.9,0.06,SW,4.5,18.5,NaN,48.0,49.8,0.0,17.16,0.10,0.13,30.18,TriCities
2,2020/04/03,3,30.2,43.9,57.3,24.0,49.4,0.01,SW,7.3,25.0,NaN,48.4,49.9,0.0,18.44,0.13,0.19,30.06,TriCities
3,2020/04/04,4,25.8,43.8,56.1,24.7,51.6,0.03,N,4.0,13.2,NaN,48.0,49.7,0.0,18.21,0.10,0.14,29.84,TriCities
4,2020/04/05,5,40.8,49.6,60.3,35.0,59.1,0.00,N,4.7,13.9,NaN,50.7,51.9,0.0,16.95,0.11,0.15,29.70,TriCities


Othello data

In [11]:
ApOtDF = pd.read_csv('/Users/davidwheeler/Desktop/RESEARCH/Data/LateBightForecast/Forecasts/30DaySpotForecasts/AprilMayRain2020/AprilRain2020/OthelloApril_2020.csv')  

In [12]:
ApOtDF['Station'] = "Othello"

In [13]:
ApOtDF.head()

,Date,Date.1,Min°F,Avg°F,Max°F,AvgDP°F,AvgRH%,AvgLWu.,AvgDir°,AvgSpeedmph,MaxGustmph,Avg 2 in.°F,Min°F.1,Avg°F.1,TotPrecin,TotalSolarRadMJ/m²,EToin,ETrin,Station
0,2020/04/01,1,26.7,37.3,49.2,28.2,72.0,0.02,W,5.7,15.7,NaN,43.0,45.1,0.0,15.50,0.08,0.10,Othello
1,2020/04/02,2,24.3,37.8,50.4,20.4,53.2,0.01,W,5.4,16.4,NaN,41.5,44.4,0.0,19.53,0.10,0.14,Othello
2,2020/04/03,3,26.7,38.8,52.0,22.4,56.7,0.00,SW,10.1,28.6,NaN,42.2,45.0,0.0,19.06,0.12,0.18,Othello
3,2020/04/04,4,25.5,41.1,52.0,22.2,51.0,0.00,NE,3.9,14.6,NaN,42.2,45.3,0.0,19.59,0.10,0.13,Othello
4,2020/04/05,5,37.5,45.3,56.2,34.4,68.3,0.00,N,5.4,15.3,NaN,45.7,47.7,0.0,14.91,0.10,0.13,Othello


Concatenate dfs

In [14]:
AprilDF = pd.concat([ApPrDF, ApTCDF, ApOtDF])

In [15]:
AprilDF

,Date,Date.1,Min°F,Avg°F,Max°F,AvgDP°F,AvgRH%,AvgLWu.,AvgDir°,AvgSpeedmph,MaxGustmph,Avg 2 in.°F,Min°F.1,Avg°F.1,TotPrecin,TotalSolarRadMJ/m²,EToin,ETrin,Station,AvgAtm.PressinHg
0,2020/04/01,1,30.1,40.2,49.9,27.8,62.0,0.01,W,4.1,17.1,NaN,48.0,49.7,0.0,13.57,0.08,0.10,Prosser,NaN
1,2020/04/02,2,26.6,40.2,53.0,25.3,59.0,0.02,SW,4.5,22.5,NaN,46.6,49.0,0.0,19.42,0.10,0.13,Prosser,NaN
2,2020/04/03,3,26.9,40.5,54.1,21.1,49.8,0.00,W,6.1,31.4,NaN,46.8,49.3,0.0,20.68,0.12,0.17,Prosser,NaN
3,2020/04/04,4,29.7,42.5,53.4,22.4,47.6,0.01,NE,4.8,24.3,NaN,47.1,49.1,0.0,16.25,0.10,0.14,Prosser,NaN
4,2020/04/05,5,38.0,47.3,58.3,32.7,58.0,0.00,NE,3.9,18.9,NaN,49.4,51.0,0.0,16.26,0.10,0.14,Prosser,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,2020/04/26,26,39.8,54.3,66.9,32.1,46.8,0.00,SW,5.8,18.2,NaN,53.6,57.2,0.0,23.53,0.17,0.24,Othello,NaN
26,2020/04/27,27,48.8,59.9,72.0,36.3,43.3,0.00,SW,13.3,32.5,NaN,56.7,59.2,0.0,24.12,0.26,0.38,Othello,NaN
27,2020/04/28,28,35.9,55.6,67.6,36.2,50.9,0.01,E,4.5,13.2,NaN,55.4,59.2,0.0,21.71,0.15,0.19,Othello,NaN
28,2020/04/29,29,46.4,60.4,73.1,41.7,52.0,0.00,NW,6.1,15.7,NaN,57.7,60.6,0.0,17.90,0.17,0.24,Othello,NaN


Number of rainy days in April

In [16]:
AprilDF.TotPrecin.groupby([AprilDF.Station]).value_counts()

Station    TotPrecin
Othello    0.00         29
           0.06          1
Prosser    0.00         29
           0.07          1
TriCities  0.00         29
           0.04          1
Name: TotPrecin, dtype: int64

#### Import data for rain in May

Packages and modules

In [17]:
import pandas as pd
import datetime

In [18]:
df = pd.read_excel (r'/Users/davidwheeler/Desktop/RESEARCH/Data/LateBightForecast/Forecasts/30DaySpotForecasts/5-16_6-21.xlsx')

In [19]:
df.head()

,30 Day Daily Rain Forecast for Three Stations in Southcentral Washington,Unnamed: 1,Unnamed: 2,,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Issued 2330 PDT 5/16/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Notes: Inputs for this forecast were from Fox...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,and MtnRT™ from WRF 6km data for days 1-6 incl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DATE,NaN,RAIN 100148 TRICITIES,NaN,NaN,NaN,RAIN 300033 PROSSER,NaN,NaN,RAIN OTHELLO 300122


Subset

In [20]:
df_simple = df.iloc[4:42,]

Rename columns

In [21]:
df_simple.columns = df_simple.iloc[0]

In [22]:
df_simple.head()

4,DATE,NaN,RAIN 100148 TRICITIES,NaN,NaN,NaN,RAIN 300033 PROSSER,NaN,NaN,RAIN OTHELLO 300122
4,DATE,NaN,RAIN 100148 TRICITIES,NaN,NaN,NaN,RAIN 300033 PROSSER,NaN,NaN,RAIN OTHELLO 300122
5,2020-05-17 00:00:00,TriCities,0.1,NaN,Prosser,NaN,0.18,NaN,Othello,0.2
6,2020-05-18 00:00:00,TriCities,0,NaN,Prosser,NaN,0.01,NaN,Othello,0
7,2020-05-19 00:00:00,TriCities,0,NaN,Prosser,NaN,0,NaN,Othello,0
8,2020-05-20 00:00:00,TriCities,0,NaN,Prosser,NaN,0,NaN,Othello,0


Remove redundant row

In [23]:
df_simple = df_simple.drop([4])

In [24]:
df_simple.drop(df_simple.tail(1).index,inplace=True)

Remove Daynmbr & NaN columns

In [25]:
df_simple.head()

4,DATE,NaN,RAIN 100148 TRICITIES,NaN,NaN,NaN,RAIN 300033 PROSSER,NaN,NaN,RAIN OTHELLO 300122
5,2020-05-17 00:00:00,TriCities,0.1,NaN,Prosser,NaN,0.18,NaN,Othello,0.2
6,2020-05-18 00:00:00,TriCities,0,NaN,Prosser,NaN,0.01,NaN,Othello,0
7,2020-05-19 00:00:00,TriCities,0,NaN,Prosser,NaN,0,NaN,Othello,0
8,2020-05-20 00:00:00,TriCities,0,NaN,Prosser,NaN,0,NaN,Othello,0
9,2020-05-21 00:00:00,TriCities,0,NaN,Prosser,NaN,0,NaN,Othello,0


Remove NaN columns

In [26]:
df_simple = df_simple.loc[:, df_simple.isnull().mean() < .8]

In [27]:
df_simple

4,DATE,NaN,RAIN 100148 TRICITIES,NaN,RAIN 300033 PROSSER,NaN,RAIN OTHELLO 300122
5,2020-05-17 00:00:00,TriCities,0.1,Prosser,0.18,Othello,0.2
6,2020-05-18 00:00:00,TriCities,0,Prosser,0.01,Othello,0
7,2020-05-19 00:00:00,TriCities,0,Prosser,0,Othello,0
8,2020-05-20 00:00:00,TriCities,0,Prosser,0,Othello,0
9,2020-05-21 00:00:00,TriCities,0,Prosser,0,Othello,0
10,2020-05-22 00:00:00,TriCities,0,Prosser,0,Othello,0
11,2020-05-23 00:00:00,TriCities,0.12,Prosser,0.18,Othello,0.1
12,2020-05-24 00:00:00,TriCities,0.15,Prosser,0.12,Othello,0.1
13,2020-05-25 00:00:00,TriCities,0,Prosser,0,Othello,0
14,2020-05-26 00:00:00,TriCities,0.15,Prosser,0.22,Othello,0.25


Remove redundant columns

In [28]:
df_simple = df_simple[["DATE","RAIN 100148 TRICITIES","RAIN 300033 PROSSER", "RAIN OTHELLO 300122"]]

Rename station columns

In [29]:
df_simple = df_simple.rename(columns={"RAIN 100148 TRICITIES": "TriCities",
                          "RAIN 300033 PROSSER": "Prosser",
                         "RAIN OTHELLO 300122": "Othello"})

In [30]:
df_simple.head()

4,DATE,TriCities,Prosser,Othello
5,2020-05-17 00:00:00,0.1,0.18,0.2
6,2020-05-18 00:00:00,0,0.01,0
7,2020-05-19 00:00:00,0,0,0
8,2020-05-20 00:00:00,0,0,0
9,2020-05-21 00:00:00,0,0,0


Remove ">=" symbols

In [31]:
cols_to_check = ['TriCities', 'Prosser', 'Othello']

In [32]:
df_simple[cols_to_check] = df_simple[cols_to_check].replace({'>=':''}, regex=True)

In [33]:
df_simple.head()

4,DATE,TriCities,Prosser,Othello
5,2020-05-17 00:00:00,0.1,0.18,0.2
6,2020-05-18 00:00:00,0.0,0.01,0.0
7,2020-05-19 00:00:00,0.0,0.00,0.0
8,2020-05-20 00:00:00,0.0,0.00,0.0
9,2020-05-21 00:00:00,0.0,0.00,0.0


Rainy days in May for each station

- convert date to datetime

In [34]:
df_simple.index = pd.to_datetime(df_simple['DATE'],format='%m/%d/%y %I:%M%p')

In [35]:
df_simple.head()

4,DATE,TriCities,Prosser,Othello
DATE,,,,
2020-05-17,2020-05-17 00:00:00,0.1,0.18,0.2
2020-05-18,2020-05-18 00:00:00,0.0,0.01,0.0
2020-05-19,2020-05-19 00:00:00,0.0,0.00,0.0
2020-05-20,2020-05-20 00:00:00,0.0,0.00,0.0
2020-05-21,2020-05-21 00:00:00,0.0,0.00,0.0


Reformat `str` as `floats` and coerce na to NaN

In [36]:
df_simple = pd.DataFrame([pd.to_numeric(df_simple[i], downcast="float", errors='coerce') for i in df_simple.columns[1:4]]).T

In [37]:
df_simple.head()

,TriCities,Prosser,Othello
DATE,,,
2020-05-17,0.1,0.18,0.2
2020-05-18,0.0,0.01,0.0
2020-05-19,0.0,0.00,0.0
2020-05-20,0.0,0.00,0.0
2020-05-21,0.0,0.00,0.0


In [38]:
df_simple

,TriCities,Prosser,Othello
DATE,,,
2020-05-17,0.10,0.18,0.20
2020-05-18,0.00,0.01,0.00
2020-05-19,0.00,0.00,0.00
2020-05-20,0.00,0.00,0.00
2020-05-21,0.00,0.00,0.00
2020-05-22,0.00,0.00,0.00
2020-05-23,0.12,0.18,0.10
2020-05-24,0.15,0.12,0.10
2020-05-25,0.00,0.00,0.00


Determine total rain per month in inches

In [39]:
df_simple.groupby([df_simple.index.month]).sum() #df_simple.groupby(pd.Grouper(freq='M')).sum() df_may = df_simple[(df_simple['DATE'] > pd.Timestamp(2020, 5, 1)) & (df_simple['DATE'] <= pd.Timestamp(2020, 5, 31))]

,TriCities,Prosser,Othello
DATE,,,
5,0.53,0.72,0.66
6,0.61,0.91,0.61


Determine number of rainy days for each station

In [40]:
df_simple.astype(bool).groupby([df_simple.index.month]).sum()

,TriCities,Prosser,Othello
DATE,,,
5,5.0,6.0,5.0
6,4.0,7.0,6.0


For TriCities and Prosser, add 2 days in May.

For Othello, add 1 day in May.

**Source of information: models**

#### Model 1: logistic regression

- **Prosser**

> $lf = 9.252 - 4.004(Y_p) - 0.660(R_{am})$

> where:

> $Y_p$: late blight outbreak during the preceding year:
$$  
Y_p
\begin{cases}
  \text{late blight absent in previous year/ no} = 0 \\    
  \text{late blight present in previous year/ yes} = 1 
\end{cases}
$$
> $R_{am}$: number of days with rain >= 0.25 mm during April and May

In [41]:
9.252 - 4.004*1 - 0.660*(1 + 9)

-1.3519999999999994

- **Tri-Cities**

> $lf = 2.987 - 3.062(Y_p) - 0.163(R_{am})$

In [42]:
2.987 - 3.062*1 - 0.163*(1 + 10)

-1.8679999999999999

- **Othello**

> $lf = 4.723 - 3.561(Y_p) - 0.293(R_{am})$

In [43]:
4.723 - 3.561*1 - 0.293*(1 + 6)

-0.8889999999999998

#### Model 2: logistic regression

- **Prosser**

> $lf = 11.513 - 3.894(Y_p) - 0.719(R_{am}) - 0.259(R_{ja})$

> where:

> $R_{ja}$: number of days with rain > = 0.25 during July and August

- **Tri-Cities**

> $lf = 1.716 - 2.844(Y_p) - 0.094(R_{ja})$

- **Othello**

> $lf = 4.960 - 3.084(Y_p) - 0.273(R_{am}) - 0.129(R_{ja})$

#### For the logistic regression models, the probability, $P$, of an outbreak is estimated:
$$
P = \frac{1}{1 + exp^{lf}}
$$

> where $lf$ is computed from equations 3 or 4.

> - if 
$$  
P
\begin{cases}
  \geq 0.5 \rightarrow \text{ outbreak year} \\    
  < 0.5 \rightarrow \text{ non-outbreak year}
\end{cases}
$$

- **Prosser**

In [44]:
1 / (1 + np.exp(-1.3519999999999994))

0.7944564113805245

- **Tri-Cities**

In [45]:
1 / (1 + np.exp(-1.8679999999999999))

0.8662266911487845

- **Othello**

In [46]:
1 / (1 + np.exp(-0.8889999999999998))

0.7086837645709135

| Factor level | Factor value |
| --- | --- |
| < 50% | 0 |
| $\geq$ 50% | 4 |

### D (date)

**Source of information: internet**

| Factor level | Factor value |
| --- | --- |
| December - April | 0 |
| May, July - November | 1 |
| June | 3J |

### E (number of rainy days expected in the next 7 days) (> 30% probability)

**Source of information: Rain forecasts from Fox Weather LLC. A rain probability of 30% has arbitrarily been used to indicate rain**

| Factor level | Factor value |
| --- | --- |
| 0 rainy days expected | 1 |
| 1 rainy days expected | 2 |
| 2 rainy days expected| 4 |
| 3 or more rainy days expected| 5 |

### Risk Factor Calculation:

**Risk** = **A** (phenology) $\cdot$ **B** (occurrence of late blight in field or adjacent fields) $\cdot$ **C** (probability of late blight occurrence in the Columbia Basin (PROB)) $\cdot$ **D** (date) $\cdot$ **E** (number of rainy days expected in the next 7 days)

- Prosser

In [1]:
1*30*4*1*1

120

- Tri-cities

In [2]:
1*30*4*1*2

240

- Othello

In [3]:
1*30*4*1*1

120

# Recommendations

**Risk factor 0:**
- Late blight is not likely, fungicide application is not recommended. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight on regular bases.


**Risk factor 1-2:**
- Late blight is not likely, fungicide application is not recommended. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.

**Risk factor 4-5:**
- Apply late blight fungicide before any rainy period. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.

**Risk factor 8 – 10:**
- Apply late blight fungicide before any rainy periods and continue for 3 wks. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.

**Risk factor 3J:**
- Apply fungicides on a 10-14 day schedule through July 4. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.

**Risk factor 6J:**
- Apply fungicides on a 10 day schedule through July 4. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.

**Risk factor 12J:**
- Apply fungicides on a 7-10 day schedule and before any rainy periods through July 4. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.

**Risk factor 15J – 30J:**
- Apply fungicides on a 7 day schedule and before any rainy periods through July 4. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.

**Risk factor 30:**
- Confirm occurrence of late blight. If present, apply late blight fungicides on a 5 to 7 day schedule and continue until harvest. Avoid over watering and irrigation during and just after rainy, cool and cloudy weather. Harvest during dry weather. Sort out rotten tubers going into storage. Consult literature on recommendations for management of late blight in the field during late season in infected tubers in storage. 